# Adding a New Benchmark to DuckDB

## Imports

In [1]:
import duckdb
import pandas as pd
import os
import time
import json
import plotly.express as px

In [2]:
scale_factors = [1, 5, 10]
tables = ["customer", "supplier", "part", "date", "lineorder"]
base_path = "./data/"

## Creating separate databases and loading the data

In [3]:
for sf in scale_factors:
    db_name = f"databases/ssb_sf{sf}.db"
    print(f"Creating database {db_name} for scale factor {sf}")
    
    if os.path.exists(db_name):
        os.remove(db_name)
    
    con = duckdb.connect(db_name)
    
    for table in tables:
        con.execute(f"DROP TABLE IF EXISTS {table}")
    
    con.execute("""
    CREATE TABLE customer (
        c_custkey INTEGER,
        c_name VARCHAR,
        c_address VARCHAR,
        c_city VARCHAR,
        c_nation VARCHAR,
        c_region VARCHAR,
        c_phone VARCHAR,
        c_mktsegment VARCHAR
    );
    """)

    con.execute("""
    CREATE TABLE supplier (
        s_suppkey INTEGER,
        s_name VARCHAR,
        s_address VARCHAR,
        s_city VARCHAR,
        s_nation VARCHAR,
        s_region VARCHAR,
        s_phone VARCHAR
    );
    """)

    con.execute("""
    CREATE TABLE part (
        p_partkey INTEGER,
        p_name VARCHAR,
        p_mfgr VARCHAR,
        p_category VARCHAR,
        p_brand VARCHAR,
        p_color VARCHAR,
        p_type VARCHAR,
        p_size INTEGER,
        p_container VARCHAR
    );
    """)

    con.execute("""
    CREATE TABLE date (
        d_datekey DATE,
        d_date VARCHAR,
        d_dayofweek VARCHAR,
        d_month VARCHAR,
        d_year INTEGER,
        d_yearmonthnum INTEGER,
        d_yearmonth VARCHAR,
        d_daynuminweek INTEGER,
        d_daynuminmonth INTEGER,
        d_daynuminyear INTEGER,
        d_monthnuminyear INTEGER,
        d_weeknuminyear INTEGER,
        d_sellingseason VARCHAR,
        d_lastdayinweekfl INTEGER,
        d_lastdayinmonthfl INTEGER,
        d_holidayfl INTEGER,
        d_weekdayfl INTEGER
    );
    """)

    con.execute("""
    CREATE TABLE lineorder (
        lo_orderkey INTEGER,
        lo_linenumber INTEGER,
        lo_custkey INTEGER,
        lo_partkey INTEGER,
        lo_suppkey INTEGER,
        lo_orderdate DATE,
        lo_orderpriority VARCHAR,
        lo_shippriority VARCHAR,
        lo_quantity INTEGER,
        lo_extendedprice INTEGER,
        lo_ordtotalprice BIGINT,
        lo_discount INTEGER,
        lo_revenue BIGINT,
        lo_supplycost BIGINT,
        lo_tax INTEGER,
        lo_commitdate DATE,
        lo_shipmode VARCHAR
    );
    """)
    
    data_path = f"{base_path}sf{sf}/"
    print(f"Processing data for scale factor: {sf}")
    
    for table in tables:
        file_path = os.path.join(data_path, f"{table}.tbl")
        
        if os.path.exists(file_path):
            con.execute(f"COPY {table} FROM '{file_path}' (DELIMITER ',', NULL '', FORMAT 'csv');")
            print(f"Loaded data into {table} table from {file_path}.")
        else:
            print(f"Data file for {table} not found at {file_path}.")
    
    con.close()
    db_size = os.path.getsize(db_name)
    print(f"Database size for scale factor {sf}: {db_size / (1024 * 1024):.0f} MB \n")

Creating database databases/ssb_sf1.db for scale factor 1
Processing data for scale factor: 1
Loaded data into customer table from ./data/sf1/customer.tbl.
Loaded data into supplier table from ./data/sf1/supplier.tbl.
Loaded data into part table from ./data/sf1/part.tbl.
Loaded data into date table from ./data/sf1/date.tbl.
Loaded data into lineorder table from ./data/sf1/lineorder.tbl.
Database size for scale factor 1: 141 MB 

Creating database databases/ssb_sf5.db for scale factor 5
Processing data for scale factor: 5
Loaded data into customer table from ./data/sf5/customer.tbl.
Loaded data into supplier table from ./data/sf5/supplier.tbl.
Loaded data into part table from ./data/sf5/part.tbl.
Loaded data into date table from ./data/sf5/date.tbl.
Loaded data into lineorder table from ./data/sf5/lineorder.tbl.
Database size for scale factor 5: 720 MB 

Creating database databases/ssb_sf10.db for scale factor 10
Processing data for scale factor: 10
Loaded data into customer table from 

In [4]:
sf_db_sizes = []
for sf in scale_factors:
    db_name = f"databases/ssb_sf{sf}.db"
    db_size = round(os.path.getsize(db_name) / (1024 * 1024), 0)
    sf_db_sizes.append({"scale_factor": sf, "size": db_size})
    
df = pd.DataFrame(sf_db_sizes)
fig = px.bar(df, x='scale_factor', y='size', text='size', labels=dict(scale_factor="Scale Factor", size="Size (MB)"))
fig.update_traces(texttemplate='%{text:.0f} MB', textposition='inside')
fig.update_xaxes(tickvals=[1,5,10])
fig.show()

## Queries (https://clickhouse.com/docs/en/getting-started/example-datasets/star-schema)

In [5]:
queries = {
    "Query1": """
        SELECT
              D_YEAR,
              S_CITY,
              P_BRAND,
              sum(LO_REVENUE - LO_SUPPLYCOST) AS profit
        FROM
              date,
              customer,
              supplier,
              part,
              lineorder
        WHERE
              LO_CUSTKEY = C_CUSTKEY
              AND LO_SUPPKEY = S_SUPPKEY
              AND LO_PARTKEY = P_PARTKEY
              AND LO_ORDERDATE = D_DATEKEY
              AND C_REGION = 'AMERICA'
              AND S_NATION = 'UNITED STATES'
              AND (D_YEAR = 1997 OR D_YEAR = 1998)
              AND P_CATEGORY = 'MFGR#14'
        GROUP BY
              D_YEAR,
              S_CITY,
              P_BRAND
        ORDER BY
              D_YEAR,
              S_CITY,
              P_BRAND
        """,
    
    "Query2": """
        SELECT
            sum(LO_EXTENDEDPRICE * LO_DISCOUNT) AS REVENUE
        FROM
            lineorder,
            date
        WHERE
            LO_ORDERDATE = D_DATEKEY
            AND D_YEARMONTHNUM = 199401
            AND LO_DISCOUNT BETWEEN 4 AND 6
            AND LO_QUANTITY BETWEEN 26 AND 35;
        """,
    
    "Query3": """
        SELECT
            C_NATION,
            S_NATION,
            D_YEAR,
            sum(LO_REVENUE) AS REVENUE
        FROM
            customer,
            lineorder,
            supplier,
            date
        WHERE
            LO_CUSTKEY = C_CUSTKEY
            AND LO_SUPPKEY = S_SUPPKEY
            AND LO_ORDERDATE = D_DATEKEY
            AND C_REGION = 'ASIA' AND S_REGION = 'ASIA'
            AND D_YEAR >= 1992 AND D_YEAR <= 1997
        GROUP BY
            C_NATION,
            S_NATION,
            D_YEAR
        ORDER BY
            D_YEAR ASC,
            REVENUE DESC;
            """
    }

## Performance Experiments

### Varying thread counts for scaling factor 10

In [6]:
sf = 10
db_name = f"databases/ssb_sf{sf}.db"
con = duckdb.connect(db_name)

num_repetitions = 5
results_dir = "results"
os.makedirs(results_dir, exist_ok=True)

print("\n--- Experiment 1: Varying thread counts --- \n")
thread_counts = [1, 4, 8]

for query_name, query in queries.items():
    for threads in thread_counts:
        file_path = os.path.join(results_dir, f"{query_name}_sf{sf}_thread_count_{threads}_all_runs.txt")
        with open(file_path, 'w') as file:
            file.write(f"Results for {query_name} with scaling factor {sf} and thread count {threads}:\n")
        
        con.execute(f"SET threads = {threads};")
        con.execute(f"PRAGMA enable_profiling = query_tree;")

        for i in range(num_repetitions):
            result = con.execute(f"EXPLAIN ANALYZE {query}").fetchall()
            formatted_result = result[0][1].replace('\n', '\n') 
            with open(file_path, 'a') as file:
                file.write(f"\nRun {i+1}:\n")
                file.write(formatted_result)
                file.write("\n")

        print(f"All results for {query_name} with scaling factor {sf} and thread count {threads} saved to '{file_path}'\n")

con.close()


--- Experiment 1: Varying thread counts --- 

All results for Query1 with scaling factor 10 and thread count 1 saved to 'results/Query1_sf10_thread_count_1_all_runs.txt'

All results for Query1 with scaling factor 10 and thread count 4 saved to 'results/Query1_sf10_thread_count_4_all_runs.txt'

All results for Query1 with scaling factor 10 and thread count 8 saved to 'results/Query1_sf10_thread_count_8_all_runs.txt'

All results for Query2 with scaling factor 10 and thread count 1 saved to 'results/Query2_sf10_thread_count_1_all_runs.txt'

All results for Query2 with scaling factor 10 and thread count 4 saved to 'results/Query2_sf10_thread_count_4_all_runs.txt'

All results for Query2 with scaling factor 10 and thread count 8 saved to 'results/Query2_sf10_thread_count_8_all_runs.txt'

All results for Query3 with scaling factor 10 and thread count 1 saved to 'results/Query3_sf10_thread_count_1_all_runs.txt'

All results for Query3 with scaling factor 10 and thread count 4 saved to 'res

### Varying scaling factors with thread count 1

In [7]:
threads = 1
scale_factors = [1, 5]
results_dir = "results"
os.makedirs(results_dir, exist_ok=True)

print("\n--- Experiment 2: Varying scaling factors --- \n")
for query_name, query in queries.items():
    for sf in scale_factors:
        db_name = f"databases/ssb_sf{sf}.db"
        con = duckdb.connect(db_name)
        con.execute(f"SET threads = {threads};")
        con.execute(f"PRAGMA enable_profiling = query_tree;")

        file_path = os.path.join(results_dir, f"{query_name}_sf{sf}_thread_count_1_all_runs.txt")
        with open(file_path, 'w') as file:
            file.write(f"Results for {query_name} with scaling factor {sf} and thread count 1:\n")

        for i in range(num_repetitions):
            result = con.execute(f"EXPLAIN ANALYZE {query}").fetchall()
            formatted_result = result[0][1].replace('\n', '\n')
            with open(file_path, 'a') as file:
                file.write(f"\nRun {i+1}:\n")
                file.write(formatted_result)
                file.write("\n")

        print(f"All results for {query_name} with scaling factor {sf} and thread count {threads} saved to '{file_path}'\n")
        con.close()


--- Experiment 2: Varying scaling factors --- 

All results for Query1 with scaling factor 1 and thread count 1 saved to 'results/Query1_sf1_thread_count_1_all_runs.txt'

All results for Query1 with scaling factor 5 and thread count 1 saved to 'results/Query1_sf5_thread_count_1_all_runs.txt'

All results for Query2 with scaling factor 1 and thread count 1 saved to 'results/Query2_sf1_thread_count_1_all_runs.txt'

All results for Query2 with scaling factor 5 and thread count 1 saved to 'results/Query2_sf5_thread_count_1_all_runs.txt'

All results for Query3 with scaling factor 1 and thread count 1 saved to 'results/Query3_sf1_thread_count_1_all_runs.txt'

All results for Query3 with scaling factor 5 and thread count 1 saved to 'results/Query3_sf5_thread_count_1_all_runs.txt'

